In [ ]:
import itk
import numpy as np
import pyvista as pv

from physiomotion4d.contour_tools import ContourTools
from physiomotion4d.convert_vtk_4d_to_usd import ConvertVTK4DToUSD
from physiomotion4d.segment_chest_total_segmentator import SegmentChestTotalSegmentator
from data_dirlab_4d_ct import DataDirLab4DCT

case_names = DataDirLab4DCT().case_names
case_names = [case_names[0]]

base_timepoint = 30

output_dir = "./results"


In [ ]:
def transform_contours_list(
    contours: pv.PolyData,
    case_name: str,
    mask_name: str,
    output_dir: str
):
    """
    Transform a list of contours to a list of transformed contours.
    """
    con_tools = ContourTools()
    new_contours = []
    for i in range(10):
        phi_FM = itk.transformread(
            f"{output_dir}/{case_name}_T{i * 10:02d}_{mask_name}_phi_FM.hdf"
        )[0]

        print(f"Transforming {case_name} - {mask_name} - T{i * 10:02d}")
        new_contours.append(con_tools.transform_contours(contours, phi_FM))

    return new_contours

In [ ]:
def make_dirlab_models(
    output_dir,
    label,
    case_name,
    base_timepoint,
    all_labelmap_arr,
    all_mask_ids,
    con_tools,
):
    """
    Make DirLab models for a list of cases.
    """
    labelmap_image = itk.imread(
        f"{output_dir}/{case_name}_T{base_timepoint}_{label}_mask_org.mha",
        pixel_type=itk.UC,
    )
    labelmap_arr = itk.array_view_from_image(labelmap_image)

    print(f"Extracting contours from {case_name} - {label} Contours")
    label_labelmap_arr = (all_labelmap_arr * labelmap_arr).astype(np.uint8)
    label_labelmap_image = itk.image_from_array(label_labelmap_arr)
    label_labelmap_image.CopyInformation(labelmap_image)

    contours = con_tools.extract_contours(label_labelmap_image)
    contours.save(
        f"{output_dir}/{case_name}_T{base_timepoint}_{label}_lungGatedBase.vtp",
        binary=True,
    )

    print(f"Applying transforms to vtp models from {case_name}")
    transformed_contours = transform_contours_list(
        contours,
        case_name,
        label,
        output_dir
    )

    print(f"Converting vtp models to USD for {case_name}")
    converter = ConvertVTK4DToUSD(
        "DirLab4DCT",
        transformed_contours,
        mask_ids=all_mask_ids,
    )
    converter.convert(
        f"{output_dir}/{case_name}_{label}_lungGated.usd",
        convert_to_surface=True,
    )

In [ ]:
con_tools = ContourTools()

seg = SegmentChestTotalSegmentator()
for case_name in case_names:
    # all labelmap
    all_labelmap = itk.imread(
        f"{output_dir}/{case_name}_T{base_timepoint}_all_mask_org.mha",
        pixel_type=itk.UC,
    )
    all_labelmap_arr = itk.array_view_from_image(all_labelmap)
    all_mask_ids = seg.all_mask_ids

    for label in ["all", "static_anatomy", "dynamic_anatomy"]:
        make_dirlab_models(
            output_dir,
            label,
            case_name,
            base_timepoint,
            all_labelmap_arr,
            all_mask_ids, con_tools
        )